In [41]:
from brightest_path_lib.cost import Cost, Reciprocal

from transonic import boost

In [42]:
@boost
class FastReciprocal(Cost):

    min_intensity: float
    max_intensity: float
    RECIPROCAL_MIN: float
    RECIPROCAL_MAX: float
    _min_step_cost: float

    def __init__(self, min_intensity: float, max_intensity: float) -> None:
        super().__init__()
        if min_intensity is None or max_intensity is None:
            raise TypeError
        if min_intensity > max_intensity:
            raise ValueError
        self.min_intensity = min_intensity
        self.max_intensity = max_intensity
        self.RECIPROCAL_MIN = 1E-6
        self.RECIPROCAL_MAX = 255.0
        self._min_step_cost = 1 / self.RECIPROCAL_MAX
        
    @boost
    def cost_of_moving_to(self, intensity_at_new_point: float) -> float:
        if intensity_at_new_point > self.max_intensity:
            raise ValueError

        intensity_at_new_point = self.RECIPROCAL_MAX * (intensity_at_new_point - self.min_intensity) / (self.max_intensity - self.min_intensity)

        if intensity_at_new_point < self.RECIPROCAL_MIN:
            intensity_at_new_point = self.RECIPROCAL_MIN
        
        return 1.0 / intensity_at_new_point
    
    @boost
    def minimum_step_cost(self) -> float:
        return self._min_step_cost


In [43]:
%%timeit -n45554

fr = FastReciprocal(0, 10000)

fr.cost_of_moving_to(100)
fr.minimum_step_cost()

1.3 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 45,554 loops each)


In [44]:
%%timeit -n45554

r = Reciprocal(0, 10000)

r.cost_of_moving_to(100)
r.minimum_step_cost()

1.35 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 45,554 loops each)
